In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas.io.json import json_normalize
import json
from matplotlib.pyplot import plot,figure
from matplotlib import pyplot as plt
import plotly.offline as offline
import plotly.graph_objs as go
import lightgbm as lgb
offline.init_notebook_mode()
import seaborn as sns
import plotly.graph_objs as go

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GroupKFold
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
json_data=["device","geoNetwork","totals","trafficSource"]

nan_list = ["not available in demo dataset",
            "unknown.unknown",
            "(not provided)",
            "(not set)", 
            "Not Socially Engaged"]
not_dic={not_val:np.nan for not_val in nan_list}
not_dic
def load_data(file):
    path="../input/"+ file
    df = pd.read_csv(path, converters = {column: json.loads for column in json_data},
                     dtype={'fullVisitorId': str, 'date': str,}, 
            parse_dates=['date'])
    for column in json_data:
        dataframe = json_normalize(df[column])
        dataframe.columns = [f"{column}_{subcolumn}" for subcolumn in dataframe.columns]
        df = df.drop(column, axis = 1).merge(dataframe, right_index = True, left_index = True)
        df.replace(not_dic, inplace=True)
    return df
train=load_data("train.csv")
test=load_data("test.csv")

In [ ]:
(train.head())

In [ ]:
missing=train.isnull().sum().sort_values(ascending=True)
missing.head()

In [ ]:
train.describe()

In [ ]:
import datetime as dt
train["day"]=train["date"].dt.weekday
train["month"]=train["date"].dt.month
train["year"]=train["date"].dt.year
train["totals_transactionRevenue"]=pd.to_numeric(train["totals_transactionRevenue"]).fillna(0)
#train['totals_hits'] = pd.to_numeric(train['totals_hits']).fillna(0)
#train['totals_pageviews'] = pd.to_numeric(train['totals_pageviews']).fillna(0)
cols_to_drop = [column for column in train.columns if train[column].nunique()==1]
train = train.drop(cols_to_drop , axis=1)


In [ ]:
plt.figure(figsize=(12,6))
revenue_by_channelgroup=train.groupby(['channelGrouping'])["totals_transactionRevenue"].mean().sort_values(ascending=False).reset_index()
#sns.barplot(x=revenue_by_channelgroup.channelGrouping,y=revenue_by_channelgroup.totals_transactionRevenue)
x_c=revenue_by_channelgroup["channelGrouping"]
y_c=revenue_by_channelgroup["totals_transactionRevenue"]
data = [go.Bar(
            x=x_c,
            y=y_c,
            textposition = 'auto',
            marker=dict(
                color='rgb(158,202,225)',
                line=dict(
                    color='rgb(8,48,107)',
                    width=1.5),
            ))]
layout = go.Layout(
    title='Channel Grouping Mean Revenue')
fig = go.Figure(data=data, layout=layout)
offline.iplot(fig, filename='basic-bar')

In [ ]:


revenue_by_channel = train.groupby(['channelGrouping','month'])['totals_transactionRevenue'].mean().reset_index()

data= {
    'data': [
        {
            'x' : revenue_by_channel[revenue_by_channel['channelGrouping']==device]['month'],
            'y': revenue_by_channel[revenue_by_channel['channelGrouping']==device]['totals_transactionRevenue'],
            'name': device
        } for device in ['Display', 'Referral', 'Direct',"Paid search","Organic Search","(Other)","Affiliates","Social"]
    ],
    'layout': {
        'title' : 'Month wise mean revenue from each Chanel Grouping',
        'xaxis': {'title': 'Month'},
        'yaxis': {'title': "Mean Revenue"}
    }
}

offline.iplot(data)



In [ ]:
plt.figure(figsize=(12,6))
revenue_by_country=train.groupby(['geoNetwork_country'])["totals_transactionRevenue"].mean().sort_values(ascending=False).reset_index()
x_country=revenue_by_country.geoNetwork_country.head(10)
y_country=revenue_by_country.totals_transactionRevenue.head(10)
#sns.barplot(x=revenue_by_country.geoNetwork_country,y=revenue_by_channelgroup.totals_transactionRevenue)
revenue_by_country.count()
data_country = [go.Bar(
            x=x_country,
            y=y_country,
            textposition = 'auto',
            marker=dict(
                color='rgb(158,202,225)',
                line=dict(
                    color='rgb(8,48,107)',
                    width=1.5),
            ))]
layout = go.Layout(
    title='Network countruy Mean Revenue')
fig_country = go.Figure(data=data_country, layout=layout)
offline.iplot(fig_country, filename='basic-bar')
#offline.iplot(data_country, filename='basic-bar')

In [ ]:
monthy_dev_rev = train.groupby(['device_deviceCategory','month'])['totals_transactionRevenue'].mean().reset_index()
offline.iplot({
    'data': [
        {
            'x' : monthy_dev_rev[monthy_dev_rev['device_deviceCategory']==device]['month'],
            'y': monthy_dev_rev[monthy_dev_rev['device_deviceCategory']==device]['totals_transactionRevenue'],
            'name': device
        } for device in ['desktop', 'mobile', 'tablet']
    ],
    'layout': {
        'title' : 'Month wise mean revenue from each device category',
        'xaxis': {'title': 'Month'},
        'yaxis': {'title': "Mean Revenue"}
    }
})

In [ ]:
total_revenue_per_coustomer=train.groupby(["fullVisitorId"])["totals_transactionRevenue"].sum().sort_values(ascending=False).reset_index()
total_revenue_per_coustomer.head()

**user level**

In [ ]:
def get_folds(df=None, n_splits=5):
    """Returns dataframe indices corresponding to Visitors Group KFold"""
    # Get sorted unique visitors
    unique_vis = np.array(sorted(df['fullVisitorId'].unique()))

    # Get folds
    folds = GroupKFold(n_splits=n_splits)
    fold_ids = []
    ids = np.arange(df.shape[0])
    for trn_vis, val_vis in folds.split(X=unique_vis, y=unique_vis, groups=unique_vis):
        fold_ids.append(
            [
                ids[df['fullVisitorId'].isin(unique_vis[trn_vis])],
                ids[df['fullVisitorId'].isin(unique_vis[val_vis])]
            ]
        )

    return fold_ids

In [ ]:
train.head()

In [ ]:
y_pred=train["totals_transactionRevenue"]

In [ ]:
k=['socialEngagementType', 'device_browserSize' ,"totals_transactionRevenue","day","month","year",'device_browserVersion', 'device_browserSize' ,'device_flashVersion','device_language', 'device_mobileDeviceBranding', 'device_mobileDeviceInfo','device_mobileDeviceMarketingName' ,'device_mobileDeviceModel','device_mobileInputSelector','device_operatingSystemVersion','device_screenColors', 'device_screenResolution', 'geoNetwork_cityId','geoNetwork_latitude','geoNetwork_longitude','trafficSource_adwordsClickInfo.adNetworkType','trafficSource_adwordsClickInfo.adNetworkType','trafficSource_adwordsClickInfo.criteriaParameters','trafficSource_adwordsClickInfo.gclId', 'trafficSource_adwordsClickInfo.slot','trafficSource_adwordsClickInfo.page']
for col in train.columns:
    if col in k:
        del train[col]
train.shape
for col in test.columns:
    if col in k:
        del test[col]


In [ ]:
train.columns,test.columns

In [ ]:
for df in [train,test]:
    df["visit_time"]=pd.to_datetime(df["visitStartTime"],unit="s")
    df["visit_date_dayofweek"]=df["visit_time"].dt.dayofweek
    df["visit_date_dateofhours"]=df["visit_time"].dt.hour
    df["visit_date_dateofmonth"]=df["visit_time"].dt.day
    df.sort_values(['fullVisitorId', 'visit_time'], ascending=True, inplace=True)
    df['next_session_1'] = (
        df['visit_time'] - df[['fullVisitorId', 'visit_time']].groupby('fullVisitorId')['visit_time'].shift(1)
    ).astype(np.int64) // 1e9 // 60 // 60
    df['next_session_2'] = (
        df['visit_time'] - df[['fullVisitorId', 'visit_time']].groupby('fullVisitorId')['visit_time'].shift(-1)
    ).astype(np.int64) // 1e9 // 60 // 60
    
   
    
train.head()

In [ ]:
train.shape

In [ ]:
excluded_features = [
    'date', 'fullVisitorId', 'sessionId', 'totals.transactionRevenue', 
    'visitId', 'visitStartTime', 'nb_sessions', 'max_visits','visit_time'
]

categorical_features = [
    _f for _f in train.columns
    if (_f not in excluded_features) & (train[_f].dtype == 'object')
]

In [ ]:
for f in categorical_features:
    train[f], indexer = pd.factorize(train[f])
    test[f] = indexer.get_indexer(test[f])

In [ ]:
train.head()

In [ ]:
folds=get_folds(df=train,n_splits=5)
train_features = [_f for _f in train.columns if _f not in excluded_features]
print(train_features)

importances = pd.DataFrame()
oof_reg_preds = np.zeros(train.shape[0])
sub_reg_preds = np.zeros(test.shape[0])
for fold_, (trn_, val_) in enumerate(folds):
    trn_x, trn_y = train[train_features].iloc[trn_], y_pred.iloc[trn_]
    val_x, val_y = train[train_features].iloc[val_], y_pred.iloc[val_]
    reg = lgb.LGBMRegressor(
        num_leaves=31,
        learning_rate=0.03,
        n_estimators=1000,
        subsample=.9,
        colsample_bytree=.9,
        random_state=1
    )
    reg.fit(
        trn_x, np.log1p(trn_y),
        eval_set=[(val_x, np.log1p(val_y))],
        early_stopping_rounds=50,
        verbose=100,
        eval_metric='rmse'
    )
    imp_df = pd.DataFrame()
    imp_df['feature'] = train_features
    imp_df['gain'] = reg.booster_.feature_importance(importance_type='gain')
    
    imp_df['fold'] = fold_ + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)
    
    oof_reg_preds[val_] = reg.predict(val_x, num_iteration=reg.best_iteration_)
    oof_reg_preds[oof_reg_preds < 0] = 0
    _preds = reg.predict(test[train_features], num_iteration=reg.best_iteration_)
    _preds[_preds < 0] = 0
    sub_reg_preds += np.expm1(_preds) / len(folds)
    
mean_squared_error(np.log1p(y_pred), oof_reg_preds) ** .5